In [ ]:
import sys, os, math, importlib
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from sklearn.covariance import GraphicalLassoCV, LedoitWolf, EmpiricalCovariance, MinCovDet, ShrunkCovariance


import torch.nn as nn
from torch.functional import F
import torch.distributions as dist
from torch.utils.data import DataLoader, TensorDataset, random_split

import pyccl as ccl
import powerbox as pbox

sys.path.append('../')
from utils_modules.models import SummaryNet, Expander
from utils_modules.vicreg import vicreg_loss
import utils_modules.data as utils_data


In [ ]:
# select device; use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: %s'%(device))

In [ ]:
# load maps and parameters used for training
maps      = np.load(...)[:, :, None, :, :]
dset_size = maps.shape[0] # data set size
splits    = maps.shape[1] # number of realizations per parameter set

params  = np.load(...)[:, None, :]
params  = np.repeat(params, splits, axis = 1) # reshape the parameters to match the shape of the maps

# pre-process the maps data set
rescale     = True
standardize = True

if rescale:
    maps = np.log(maps+1)
if standardize:
    maps_mean, maps_std = np.mean(maps, dtype=np.float64), np.std(maps, dtype=np.float64)
    maps = (maps - maps_mean)/maps_std
    
if verbose:
    print('Shape of parameters and maps:', params.shape, maps.shape)
    print('Parameter 1 range of values: [{:.3f}, {:.3f}]'.format(params[:, :, 0].min(), params[:, :, 0].max()))
    print('Parameter 2 range of values: [{:.3f}, {:.3f}]'.format(params[:, :, 1].min(), params[:, :, 1].max()))
    
    if rescale: print('Rescale: ', rescale)
    if standardize: print('Standardize: ', standardize)

In [ ]:
# load the encoder model
num_params = 2 
num_tril = int(num_params * (num_params + 1) / 2)  # Number of parameters in lower triangular matrix, for symmetric matrix
num_out = num_params + num_tril  # Dummy output of neural network

hidden = 8 
last_layer = 2*hidden

inv, var, cov = 5, 5, 1
fmodel = save_dir + 'model_{:d}_{:d}_{:d}_{:d}k_samples.pt'.format(inv, var, cov, num_sims_k)
fout   = save_dir + 'losses_{:d}_{:d}_{:d}_{:d}k_samples.txt'.format(inv, var, cov, num_sims_k)

model_encoder = SummaryNet(hidden = hidden, last_layer = last_layer).to(device)
model_encoder.load_state_dict(torch.load(fmodel))
model_encoder.eval();

In [6]:
def get_maps_arr(params, splits, 
                 maps_mean, maps_std,
                 BoxSize = 1000.0, Npixel = 100):
    
    OmegaM = params[0]
    sigma8 = params[1]

    OmegaB = 0.05
    OmegaC = OmegaM - OmegaB
    h    = 0.7
    ns   = 0.96
    
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                          h=h, sigma8 = sigma8, n_s=ns, 
                          transfer_function='eisenstein_hu')
    
    dfs_2D_splits = []
    
    for j in range(splits):
        # generate a 2D Gaussian field
        pb = pbox.PowerBox(
            N=Npixel,                     
            dim=2,                        
            pk = lambda k_val: ccl.linear_matter_power(cosmo_ccl, k_val, 1.0)/BoxSize, 
            boxlength = BoxSize,           
            seed = j,                
        )
        
        # convert it to a lognormal field
        delta_g = pb.delta_x()
        var_g = np.var(delta_g)
        rho_ln = np.exp(delta_g - var_g/2)
        
        dfs_2D_splits.append(rho_ln - 1)
        
    dfs_2D = np.array(dfs_2D_splits)[:, None, :, :]
    dfs_2D = np.log(dfs_2D+1)
    dfs_2D = (dfs_2D - maps_mean)/maps_std
    return dfs_2D

In [9]:
def get_delta_X(delta, params, net, index, splits, maps_mean, maps_std):
    
    delta_A = delta
    params_delta_A = params.copy()
    params_delta_A[index] = params_delta_A[index] + delta_A
    maps_delta_A = get_maps_arr(params_delta_A, splits, maps_mean, maps_std)
    
    params_2delta_A = params.copy()
    params_2delta_A[index] = params_2delta_A[index] + 2*delta_A
    maps_2delta_A = get_maps_arr(params_2delta_A, splits, maps_mean, maps_std) 
    
    params_neg_delta_A = params.copy()
    params_neg_delta_A[index] = params_neg_delta_A[index] - delta_A
    maps_neg_delta_A = get_maps_arr(params_neg_delta_A, splits, maps_mean, maps_std)

    params_neg_2delta_A = params.copy()
    params_neg_2delta_A[index] = params_neg_2delta_A[index] - 2*delta_A
    maps_neg_2delta_A = get_maps_arr(params_neg_2delta_A, splits, maps_mean, maps_std)
    
    net.eval()
    with torch.no_grad():     
        x = torch.tensor(maps_delta_A).float().to(device) 
        representations_delta_A = net(x).cpu().numpy()

        x = torch.tensor(maps_2delta_A).float().to(device) 
        representations_2delta_A = net(x).cpu().numpy()

        x = torch.tensor(maps_neg_delta_A).float().to(device) 
        representations_neg_delta_A = net(x).cpu().numpy()

        x = torch.tensor(maps_neg_2delta_A).float().to(device) 
        representations_neg_2delta_A = net(x).cpu().numpy()
 
    deriv_A = (-representations_2delta_A + 8*representations_delta_A - 8*representations_neg_delta_A + representations_neg_2delta_A)/(12*delta_A)
    return deriv_A.mean(axis = 0)

In [ ]:
# set fiducial cosmological parameters
params = np.array([0.3, 0.8])
param_A = params[0]
param_B = params[1]

# generate maps to estimate covariance matrix for VICReg summaries
dfs_2D = get_maps_arr(params, splits = 10000, 
                      maps_mean = maps_mean, maps_std = maps_std)
# compute summaries of the map
model_encoder.eval()
with torch.no_grad(): 
    x = torch.tensor(dfs_2D).float().to(device)
    representations = model_encoder(x) 

# compute covariance matrix
representations_arr = representations.cpu().numpy()
representations_bar = representations_arr.mean(axis = 0)

delta_representations = (representations_arr - representations_bar)

model = LedoitWolf()
model.fit(delta_representations)
lw_cov_ = model.covariance_
lw_prec_ = model.precision_

In [25]:
# compute derivatives of the summaries with respect to OmegaM, sigma8
deriv_A = get_delta_X(0.005*param_A, params, model_encoder, index = 0, splits = 1000,
                     maps_mean = maps_mean, maps_std = maps_std)
deriv_B = get_delta_X(0.005*param_B, params, model_encoder, index = 1, splits = 1000,
                     maps_mean = maps_mean, maps_std = maps_std)

In [ ]:
D_AA = 2* (deriv_A[:, None]@deriv_A[None, :])
D_BB = 2* (deriv_B[:, None]@deriv_B[None, :])
D_AB = (deriv_A[:, None]@deriv_B[None, :] + deriv_B[:, None]@deriv_A[None, :])

F_comp_AA = (1/2)*np.trace(lw_prec_@D_AA)
F_comp_BB = (1/2)*np.trace(lw_prec_@D_BB)
F_comp_AB = (1/2)*np.trace(lw_prec_@D_AB)

F_comp = np.array([[F_comp_AA, F_comp_AB], [F_comp_AB, F_comp_BB]])
print(F_comp, np.linalg.det(F_comp))
F_comp_inv = np.linalg.inv(F_comp)
stdevA, stdevB = np.sqrt(F_comp_inv[0, 0]), np.sqrt(F_comp_inv[1, 1])

print('Relative Errors on A, B in %: ', stdevA/param_A*100, stdevB/param_B*100)
print('\nFisher matrix: \n', F_comp, '\n \nInverse Fisher matrix: \n', F_inv)
print('\nFisher information: ', np.log(np.linalg.det(F_comp))/2)